In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Aug 23 12:22:38 2023

@author: vishalr
"""
import numpy as np
import pandas as pd
import pickle

# change this to the folder where you store your data
data_dir = r"/content/sample_data/"

# each of the two data frames below have 20,000 rows, each corresponding to one sample from the original graph
# each sample consists of 21 nodes; node_labels contains the names of these 21 nodes of the form Vxyz
# each node is a DNA fragment of length 500 bases; so Vxyz coveres region [500*xyz, 500*xyz+500)
# adjacency_matrix has the flattended adjacecny matrix for each of these 20,000 samples
# so each row is of dimension 21*21 = 441

# node_labels = pd.read_pickle(data_dir+'df_chr2L_drosophila_ChIA_Drop_0.1_PASS_20000_MCMC_pivot_sample_node_matrix')
# adjacency_matrix = pd.read_pickle(data_dir+'df_chr2L_drosophila_ChIA_Drop_0.1_PASS_20000_MCMC_pivot')

node_labels_chr2L = np.load(data_dir + 'df_chr2L_drosophila_ChIA_Drop_0.1_PASS_20000_MCMC_pivot_sample_node_matrix.npy')
adjacency_matrix_chr2L = np.load(data_dir + 'df_chr2L_drosophila_ChIA_Drop_0.1_PASS_20000_MCMC_pivot.npy')

node_labels_chr2R = np.load(data_dir + 'df_chr2R_drosophila_ChIA_Drop_0.1_PASS_20000_MCMC_pivot_sample_node_matrix.npy')
adjacency_matrix_chr2R = np.load(data_dir + 'df_chr2R_drosophila_ChIA_Drop_0.1_PASS_20000_MCMC_pivot.npy')

node_labels_chr3L = np.load(data_dir + 'df_chr3L_drosophila_ChIA_Drop_0.1_PASS_20000_MCMC_pivot_sample_node_matrix.npy')
adjacency_matrix_chr3L = np.load(data_dir + 'df_chr3L_drosophila_ChIA_Drop_0.1_PASS_20000_MCMC_pivot.npy')

node_labels_chr3R = np.load(data_dir + 'df_chr3R_drosophila_ChIA_Drop_0.1_PASS_20000_MCMC_pivot_sample_node_matrix.npy')
adjacency_matrix_chr3R = np.load(data_dir + 'df_chr3R_drosophila_ChIA_Drop_0.1_PASS_20000_MCMC_pivot.npy')

# just seeing the size of input; doesn't really do anything
print("chr2L node labels shape:", node_labels_chr2L.shape)
print("chr2L adjacency matrix shape:", adjacency_matrix_chr2L.shape)
print("chr2R node labels shape:", node_labels_chr2R.shape)
print("chr2R adjacency matrix shape:", adjacency_matrix_chr2R.shape)
print("chr3L node labels shape:", node_labels_chr3L.shape)
print("chr3L adjacency matrix shape:", adjacency_matrix_chr3L.shape)
print("chr3R node labels shape:", node_labels_chr3R.shape)
print("chr3R adjacency matrix shape:", adjacency_matrix_chr3R.shape)


chr2L node labels shape: (20000, 21)
chr2L adjacency matrix shape: (20000, 442)
chr2R node labels shape: (20000, 21)
chr2R adjacency matrix shape: (20000, 442)
chr3L node labels shape: (20000, 21)
chr3L adjacency matrix shape: (20000, 442)
chr3R node labels shape: (20000, 21)
chr3R adjacency matrix shape: (20000, 442)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [2]:
# We assume that PyTorch is already installed
import torch
torchversion = torch.__version__

# Install PyTorch Scatter, PyTorch Sparse, and PyTorch Geometric
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-{torchversion}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-{torchversion}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

# Visualization
import networkx as nx
import matplotlib.pyplot as plt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 50.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
# trying print the shape of the loaded array to get the number of samples
print(f'Number of samples: {adjacency_matrix_chr2L.shape[0]}')

Number of samples: 20000


In [4]:
#part1a: delete repeated node in adjacency matrix
import numpy as np

def remove_duplicates(node_labels, adj_matrix):
    unique_nodes = []
    unique_indices = []

    for idx, node in enumerate(node_labels):
        if node not in unique_nodes:
            unique_nodes.append(node)
            unique_indices.append(idx)

    unique_nodes = np.array(unique_nodes)
    sample_adj_matrix = adj_matrix[np.ix_(unique_indices, unique_indices)]

    return unique_nodes, sample_adj_matrix

# test whether it is functioning
sample_node_labels = np.array([1, 2, 2, 3, 4, 4])
sample_adj_matrix = np.array([
    [0, 1, 1, 0, 0, 0],
    [1, 0, 1, 0, 0, 0],
    [1, 1, 0, 1, 0, 0],
    [0, 0, 1, 0, 1, 1],
    [0, 0, 0, 1, 0, 1],
    [0, 0, 0, 1, 1, 0]
])

cleaned_nodes, cleaned_adj_matrix = remove_duplicates(sample_node_labels, sample_adj_matrix)

print("Cleaned Nodes:", cleaned_nodes)
print("Cleaned Adjacency Matrix:\n", cleaned_adj_matrix)

Cleaned Nodes: [1 2 3 4]
Cleaned Adjacency Matrix:
 [[0 1 0 0]
 [1 0 0 0]
 [0 0 0 1]
 [0 0 1 0]]


In [5]:
import numpy as np

# Define the total length of the array
total_length = 240000

# Create an array with values from 0 to 239999
all_node_indices = np.arange(total_length)

# Print to check the result
print("All node indices array:", all_node_indices)
print("Shape of all node indices array:", all_node_indices.shape)


All node indices array: [     0      1      2 ... 239997 239998 239999]
Shape of all node indices array: (240000,)


In [6]:
# part2b：indicate nodes' class
file_indices = []

section_length = 60000
num_sections = 4

for section in range(num_sections):
    file_indices.extend([section] * section_length)

file_indices = np.array(file_indices)

# Print to check the result
print("File indices array:", file_indices)
print("Shape of file indices array:", file_indices.shape)

# Print each section to verify
print("First section (should be 0):")
print(file_indices[:section_length])

print("Second section (should be 1):")
print(file_indices[section_length:2 * section_length])

print("Third section (should be 2):")
print(file_indices[2 * section_length:3 * section_length])

print("Fourth section (should be 3):")
print(file_indices[3 * section_length:])


File indices array: [0 0 0 ... 3 3 3]
Shape of file indices array: (240000,)
First section (should be 0):
[0 0 0 ... 0 0 0]
Second section (should be 1):
[1 1 1 ... 1 1 1]
Third section (should be 2):
[2 2 2 ... 2 2 2]
Fourth section (should be 3):
[3 3 3 ... 3 3 3]


In [7]:
#part 2c: edge list
import numpy as np
import scipy.sparse as sp

def extract_edges_from_subgraph(adjacency_matrix):
    num_nodes = 21
    edges = []

    for i in range(adjacency_matrix.shape[0]):
        # Reshape the adjacency matrix
        adj_matrix = adjacency_matrix[i, :num_nodes*num_nodes].reshape((num_nodes, num_nodes))

        for row in range(num_nodes):
            for col in range(row + 1, num_nodes):
                if adj_matrix[row, col] == 1:  # If there is an edge
                    edges.append(np.array([row, col]))

    return edges

# Extract edges from the adjacency matrices
edges_chr2L = extract_edges_from_subgraph(adjacency_matrix_chr2L)
edges_chr2R = extract_edges_from_subgraph(adjacency_matrix_chr2R)
edges_chr3L = extract_edges_from_subgraph(adjacency_matrix_chr3L)
edges_chr3R = extract_edges_from_subgraph(adjacency_matrix_chr3R)

# Print results to verify
print("Edges from chr2L:", edges_chr2L[:10])  # Print the first 10 edges as an example
print("Total edges in chr2L:", len(edges_chr2L))
print("Edges from chr2R:", edges_chr2R[:10])
print("Total edges in chr2R:", len(edges_chr2R))
print("Edges from chr3L:", edges_chr3L[:10])
print("Total edges in chr3L:", len(edges_chr3L))
print("Edges from chr3R:", edges_chr3R[:10])
print("Total edges in chr3R:", len(edges_chr3R))

all_edges = edges_chr2L + edges_chr2R + edges_chr3L + edges_chr3R

# Print the total number of edges
print("Total edges across all subgraphs:", len(all_edges))

# Save edges to a file
edges_file = 'edges.npy'
np.save(edges_file, all_edges)

# Load edges from the file to verify
loaded_edges = np.load(edges_file, allow_pickle=True)
print(f"Total edges loaded: {len(loaded_edges)}")
print(f"First 10 edges: {loaded_edges[:10]}")

Edges from chr2L: [array([0, 1]), array([0, 2]), array([1, 2]), array([2, 3]), array([3, 4]), array([4, 5]), array([4, 7]), array([4, 9]), array([ 4, 11]), array([5, 6])]
Total edges in chr2L: 722875
Edges from chr2R: [array([0, 1]), array([1, 2]), array([1, 4]), array([1, 7]), array([1, 9]), array([2, 3]), array([2, 5]), array([2, 6]), array([2, 8]), array([ 2, 10])]
Total edges in chr2R: 754087
Edges from chr3L: [array([0, 1]), array([1, 2]), array([2, 3]), array([2, 5]), array([2, 6]), array([2, 8]), array([3, 4]), array([3, 6]), array([4, 5]), array([5, 6])]
Total edges in chr3L: 679556
Edges from chr3R: [array([0, 1]), array([1, 2]), array([1, 7]), array([1, 8]), array([2, 3]), array([2, 5]), array([2, 7]), array([3, 4]), array([3, 6]), array([3, 7])]
Total edges in chr3R: 718848
Total edges across all subgraphs: 2875366
Total edges loaded: 2875366
First 10 edges: [[ 0  1]
 [ 0  2]
 [ 1  2]
 [ 2  3]
 [ 3  4]
 [ 4  5]
 [ 4  7]
 [ 4  9]
 [ 4 11]
 [ 5  6]]


In [8]:
# part 2d: relate edge to subgraph
import numpy as np

def extract_edges_and_indices(adjacency_matrix, class_offset):
    num_nodes = 21
    edges = []
    subgraph_indices = []

    for subgraph_idx in range(adjacency_matrix.shape[0]):
        adj_matrix = adjacency_matrix[subgraph_idx, :num_nodes*num_nodes].reshape((num_nodes, num_nodes))

        for row in range(num_nodes):
            for col in range(row + 1, num_nodes):
                if adj_matrix[row, col] == 1:  # If there is an edge
                    edges.append(np.array([row, col]))
                    subgraph_indices.append(class_offset + subgraph_idx)  # Add class offset to subgraph index

    return edges, subgraph_indices

# Load the subgraph files
data_dir = '/content/sample_data/'
adjacency_matrix_chr2L = np.load(data_dir + 'df_chr2L_drosophila_ChIA_Drop_0.1_PASS_20000_MCMC_pivot.npy')
adjacency_matrix_chr2R = np.load(data_dir + 'df_chr2R_drosophila_ChIA_Drop_0.1_PASS_20000_MCMC_pivot.npy')
adjacency_matrix_chr3L = np.load(data_dir + 'df_chr3L_drosophila_ChIA_Drop_0.1_PASS_20000_MCMC_pivot.npy')
adjacency_matrix_chr3R = np.load(data_dir + 'df_chr3R_drosophila_ChIA_Drop_0.1_PASS_20000_MCMC_pivot.npy')

# Define offsets for each class
class_offsets = [0, 20000, 40000, 60000]

# Extract edges and subgraph indices from the subgraph files
edges_chr2L, subgraph_indices_chr2L = extract_edges_and_indices(adjacency_matrix_chr2L, class_offsets[0])
edges_chr2R, subgraph_indices_chr2R = extract_edges_and_indices(adjacency_matrix_chr2R, class_offsets[1])
edges_chr3L, subgraph_indices_chr3L = extract_edges_and_indices(adjacency_matrix_chr3L, class_offsets[2])
edges_chr3R, subgraph_indices_chr3R = extract_edges_and_indices(adjacency_matrix_chr3R, class_offsets[3])

# Combine all edges and subgraph indices
all_edges = edges_chr2L + edges_chr2R + edges_chr3L + edges_chr3R
all_subgraph_indices = subgraph_indices_chr2L + subgraph_indices_chr2R + subgraph_indices_chr3L + subgraph_indices_chr3R

# Save edges and subgraph indices to files
edges_file = '/content/edges.npy'
subgraph_indices_file = '/content/subgraph_indices.npy'

np.save(edges_file, all_edges)
np.save(subgraph_indices_file, all_subgraph_indices)

# Load edges and subgraph indices from the files to verify
loaded_edges = np.load(edges_file, allow_pickle=True)
loaded_subgraph_indices = np.load(subgraph_indices_file, allow_pickle=True)

print(f"Total edges loaded: {len(loaded_edges)}")
print(f"Total subgraph indices loaded: {len(loaded_subgraph_indices)}")

print(f"First 10 subgraph indices: {loaded_subgraph_indices[:10]}")
print(f"Last 10 subgraph indices: {loaded_subgraph_indices[-10:]}")



Total edges loaded: 2875366
Total subgraph indices loaded: 2875366
First 10 subgraph indices: [0 0 0 0 0 0 0 0 0 0]
Last 10 subgraph indices: [79999 79999 79999 79999 79999 79999 79999 79999 79999 79999]


In [9]:
#Part 2e: class of subgraphs
import numpy as np

num_subgraphs_per_class = 20000
num_classes = 4
subgraph_classes = np.concatenate([np.full(num_subgraphs_per_class, i) for i in range(num_classes)])

subgraph_classes_file = '/content/subgraph_classes.npy'

np.save(subgraph_classes_file, subgraph_classes)

# Load subgraph classes from the file to verify
loaded_subgraph_classes = np.load(subgraph_classes_file, allow_pickle=True)

print(f"Shape of result: {loaded_subgraph_classes.shape}")

print(f"First 10 subgraph classes: {loaded_subgraph_classes[:10]}")
print(f"Last 10 subgraph classes: {loaded_subgraph_classes[-10:]}")

import numpy as np

task_2d_file = 'subgraph_indices.npy'
task_2d = np.load(task_2d_file, allow_pickle=True)

num_subgraphs_per_class = 20000
num_classes = 4

task_2e = (task_2d // num_subgraphs_per_class).flatten()

task_2e_file = 'task_2e.npy'
np.save(task_2e_file, task_2e)

loaded_task_2e = np.load(task_2e_file, allow_pickle=True)

print("Loaded Task 2E: ")
print(loaded_task_2e)
print("Shape of loaded Task 2E: ")
print(loaded_task_2e.shape)
print("First 10 edge classes: ", loaded_task_2e[:10])
print("Last 10 edge classes: ", loaded_task_2e[-10:])


Shape of result: (80000,)
First 10 subgraph classes: [0 0 0 0 0 0 0 0 0 0]
Last 10 subgraph classes: [3 3 3 3 3 3 3 3 3 3]
Loaded Task 2E: 
[0 0 0 ... 3 3 3]
Shape of loaded Task 2E: 
(2875366,)
First 10 edge classes:  [0 0 0 0 0 0 0 0 0 0]
Last 10 edge classes:  [3 3 3 3 3 3 3 3 3 3]


In [11]:
#introducing frobenius norm
import numpy as np

def frobenius_norm(mat):
    return np.linalg.norm(mat, 'fro')


def normalized_frobenius_norm(mat, ref):
    size = mat.size
    diff = mat - ref
    return frobenius_norm(diff) / np.sqrt(size)


def create_reference_matrix(size):
    ref = np.zeros((size, size))
    for i in range(size - 1):
        ref[i, i + 1] = 1
        ref[i + 1, i] = 1
    return ref

In [17]:
import torch
from torch_geometric.data import Data, InMemoryDataset, DataLoader
from torch_geometric.utils import from_scipy_sparse_matrix
import scipy.sparse as sp
import numpy as np

# Create reference matrix
def create_reference_matrix(size):
    ref = np.zeros((size, size))
    for i in range(size - 1):
        ref[i, i + 1] = 1
        ref[i + 1, i] = 1
    return ref

# Compute Frobenius norm
def frobenius_norm(mat):
    return np.linalg.norm(mat, 'fro')

# Compute normalized Frobenius norm
def normalized_frobenius_norm(mat, ref):
    size = mat.size
    diff = mat - ref
    return frobenius_norm(diff) / np.sqrt(size)

# Remove duplicate nodes
def remove_duplicates(node_labels, adj_matrix):
    unique_nodes = []
    unique_indices = []

    for idx, node in enumerate(node_labels):
        if node not in unique_nodes:
            unique_nodes.append(node)
            unique_indices.append(idx)

    unique_nodes = np.array(unique_nodes)
    sample_adj_matrix = adj_matrix[np.ix_(unique_indices, unique_indices)]

    return unique_nodes, sample_adj_matrix

# Create graph data
def create_graph_data(adjacency_matrix, node_labels, label_number, threshold=0.1):
    graphs = []
    for i in range(adjacency_matrix.shape[0]):
        row = adjacency_matrix[i]
        labels = node_labels[i]
        assert len(row) == 21 * 21 + 1, f"Each row should have 21*21 + 1 elements, but got {len(row)} elements."
        row = row[:-1]  # remove -1 row
        assert len(row) == 21 * 21, f"Each row should have 21*21 elements after removing -1 column, but got {len(row)} elements."
        adj_matrix = row.reshape((21, 21))

        # Remove duplicates
        labels, adj_matrix = remove_duplicates(labels, adj_matrix)

        # Create reference matrix
        reference_matrix = create_reference_matrix(adj_matrix.shape[0])

        # Compute normalized Frobenius norm
        norm = normalized_frobenius_norm(adj_matrix, reference_matrix)

        # Keep only matrices that differ significantly from the reference matrix
        if norm > threshold:
            edge_index, edge_attr = from_scipy_sparse_matrix(sp.coo_matrix(adj_matrix))
            x = torch.tensor(labels, dtype=torch.long).view(-1, 1)  # Change to long tensor
            y = torch.tensor([label_number], dtype=torch.long)
            graph_data = Data(x=x, edge_index=edge_index, y=y)
            graphs.append(graph_data)
    return graphs

worklist_node_labels = [node_labels_chr2L, node_labels_chr2R, node_labels_chr3L, node_labels_chr3R]
worklist_adjacency_matrix = [adjacency_matrix_chr2L, adjacency_matrix_chr2R, adjacency_matrix_chr3L, adjacency_matrix_chr3R]

# Process data and filter graphs that meet the criteria
dataset = []
for label_number in range(len(worklist_adjacency_matrix)):
    adj_matrix = worklist_adjacency_matrix[label_number]
    node_labels = worklist_node_labels[label_number]
    graphs = create_graph_data(adj_matrix, node_labels, label_number, threshold=0.1)
    dataset.extend(graphs)

# Create custom dataset class
class MyDataset(InMemoryDataset):
    def __init__(self, data_list, transform=None, pre_transform=None):
        self.data_list = data_list
        super(MyDataset, self).__init__(None, transform, pre_transform)
        self.data, self.slices = self.collate(data_list)

    def len(self):
        return len(self.data_list)

    def get(self, idx):
        return self.data_list[idx]

my_dataset = MyDataset(dataset)

# Get number of features and classes
num_node_features = my_dataset[0].x.size(1)
num_classes = len(set([data.y.item() for data in my_dataset]))

# Split dataset
train_dataset = my_dataset[:int(len(my_dataset) * 0.8)]
val_dataset = my_dataset[int(len(my_dataset) * 0.8):int(len(my_dataset) * 0.9)]
test_dataset = my_dataset[int(len(my_dataset) * 0.9):]

print(f'Training set   = {len(train_dataset)} graphs')
print(f'Validation set = {len(val_dataset)} graphs')
print(f'Test set       = {len(test_dataset)} graphs')

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


print('\nTrain loader:')
for i, subgraph in enumerate(train_loader):
    print(f' - Subgraph {i}:')
    print(f'  Node labels (x): {subgraph.x}')
    print(f'  Labels (y): {subgraph.y}')
    break  # Print only the first batch

print('\nValidation loader:')
for i, subgraph in enumerate(val_loader):
    print(f' - Subgraph {i}:')
    print(f'  Node labels (x): {subgraph.x}')
    print(f'  Labels (y): {subgraph.y}')
    break  # Print only the first batch

print('\nTest loader:')
for i, subgraph in enumerate(test_loader):
    print(f' - Subgraph {i}:')
    print(f'  Node labels (x): {subgraph.x}')
    print(f'  Labels (y): {subgraph.y}')
    break  # Print only the first batch

my_dataset = MyDataset(dataset)





Training set   = 63348 graphs
Validation set = 7919 graphs
Test set       = 7919 graphs

Train loader:
 - Subgraph 0:
  Node labels (x): tensor([[28115],
        [33603],
        [ 1530],
        ...,
        [ 1267],
        [32681],
        [24223]])
  Labels (y): tensor([2, 2, 1, 0, 0, 1, 1, 0, 0, 0, 0, 2, 1, 1, 1, 0, 1, 1, 0, 0, 2, 2, 0, 0,
        1, 0, 3, 1, 2, 0, 2, 3, 2, 1, 1, 0, 1, 3, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2,
        0, 3, 1, 1, 1, 2, 0, 1, 0, 1, 1, 1, 0, 0, 1, 2])

Validation loader:
 - Subgraph 0:
  Node labels (x): tensor([[33839],
        [33935],
        [33954],
        ...,
        [18945],
        [14769],
        [14784]])
  Labels (y): tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

Test loader:
 - Subgraph 0:
  Node labels (x): tensor([[43563],
        [51217],
        [51179],
        ...,
   

In [18]:
import torch
import torch.nn.functional as F
from torch.nn import Linear, Sequential, BatchNorm1d, ReLU, Dropout
from torch_geometric.nn import GCNConv, GINConv
from torch_geometric.nn import global_mean_pool, global_add_pool


# class GCN(torch.nn.Module):
#     """GCN"""
#     def __init__(self, dim_h):
#         super(GCN, self).__init__()
#         self.conv1 = GCNConv(dataset.num_node_features, dim_h)
#         self.conv2 = GCNConv(dim_h, dim_h)
#         self.conv3 = GCNConv(dim_h, dim_h)
#         self.lin = Linear(dim_h, dataset.num_classes)

#     def forward(self, x, edge_index, batch):
#         # Node embeddings
#         h = self.conv1(x, edge_index)
#         h = h.relu()
#         h = self.conv2(h, edge_index)
#         h = h.relu()
#         h = self.conv3(h, edge_index)

#         # Graph-level readout
#         hG = global_mean_pool(h, batch)

#         # Classifier
#         h = F.dropout(hG, p=0.5, training=self.training)
#         h = self.lin(h)

#         return hG, F.log_softmax(h, dim=1)

class GIN(torch.nn.Module):
    """GIN"""
    def __init__(self, dim_h):
        super(GIN, self).__init__()
        self.conv1 = GINConv(
            Sequential(Linear(my_dataset.num_node_features, dim_h),
                       BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()))
        self.conv2 = GINConv(
            Sequential(Linear(dim_h, dim_h), BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()))
        self.conv3 = GINConv(
            Sequential(Linear(dim_h, dim_h), BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()))
        self.lin1 = Linear(dim_h*3, dim_h*3)
        self.lin2 = Linear(dim_h*3, my_dataset.num_classes)

    def forward(self, x, edge_index, batch):
        # Node embeddings
        h1 = self.conv1(x, edge_index)
        h2 = self.conv2(h1, edge_index)
        h3 = self.conv3(h2, edge_index)

        # Graph-level readout
        h1 = global_add_pool(h1, batch)
        h2 = global_add_pool(h2, batch)
        h3 = global_add_pool(h3, batch)

        # Concatenate graph embeddings
        h = torch.cat((h1, h2, h3), dim=1)

        # Classifier
        h = self.lin1(h)
        h = h.relu()
        h = F.dropout(h, p=0.5, training=self.training)
        h = self.lin2(h)

        return h, F.log_softmax(h, dim=1)

gin = GIN(dim_h=32)

In [19]:
def train(model, loader):
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),
                                      lr=0.01,
                                      weight_decay=0.01)
    epochs = 5

    model.train()
    for epoch in range(epochs+1):
        #print(epoch)
        total_loss = 0
        acc = 0
        val_loss = 0
        val_acc = 0

        # Train on batches
        for data in loader:
          optimizer.zero_grad()
          _, out = model(data.x, data.edge_index, data.batch)
          loss = criterion(out, data.y)
          total_loss += loss / len(loader)
          acc += accuracy(out.argmax(dim=1), data.y) / len(loader)
          loss.backward()
          optimizer.step()

          # Validation
          val_loss, val_acc = test(model, val_loader)

          # print per epoch to see if the accuracy goes up!
        print(f'Epoch {epoch:>0} | Train Loss: {total_loss:.2f} '
            f'| Train Acc: {acc*100:>5.2f}% '
            f'| Val Loss: {val_loss:.2f} '
            f'| Val Acc: {val_acc*100:.2f}%')


    test_loss, test_acc = test(model, test_loader)
    print(f'Test Loss: {test_loss:.2f} | Test Acc: {test_acc*100:.2f}%')

    return model

@torch.no_grad()
def test(model, loader):
    criterion = torch.nn.CrossEntropyLoss()
    model.eval()
    loss = 0
    acc = 0

    for data in loader:
        _, out = model(data.x, data.edge_index, data.batch)
        loss += criterion(out, data.y) / len(loader)
        acc += accuracy(out.argmax(dim=1), data.y) / len(loader)

    return loss, acc

def accuracy(pred_y, y):
    """Calculate accuracy."""
    return ((pred_y == y).sum() / len(y)).item()

gin = train(gin, train_loader)

Epoch 0 | Train Loss: 1.23 | Train Acc: 38.50% | Val Loss: 3.05 | Val Acc: 0.00%
Epoch 1 | Train Loss: 1.11 | Train Acc: 45.85% | Val Loss: 2.21 | Val Acc: 17.35%
Epoch 2 | Train Loss: 1.09 | Train Acc: 47.04% | Val Loss: 1.58 | Val Acc: 41.10%
Epoch 3 | Train Loss: 1.09 | Train Acc: 47.18% | Val Loss: 1.27 | Val Acc: 54.03%
Epoch 4 | Train Loss: 1.09 | Train Acc: 47.32% | Val Loss: 1.54 | Val Acc: 42.37%
Epoch 5 | Train Loss: 1.09 | Train Acc: 47.25% | Val Loss: 1.97 | Val Acc: 27.62%
Test Loss: 1.93 | Test Acc: 28.76%


In [ ]:

label_mapping = {0: '2L', 1: '2R', 2: '3L', 3: '3R'}


import matplotlib.pyplot as plt
import networkx as nx
from torch_geometric.utils import to_networkx


fig, ax = plt.subplots(4, 4, figsize=(16, 16))
fig.suptitle('GIN - Graph classification')

for i, data in enumerate(dataset[1113-16:]):
    _, out = gin(data.x, data.edge_index, data.batch)
    predicted_class = out.argmax(dim=1).item()
    actual_class = data.y.item()
    color = "green" if predicted_class == actual_class else "red"

    ix = np.unravel_index(i, ax.shape)
    ax[ix].axis('off')
    G = to_networkx(dataset[1113-16+i], to_undirected=True)
    nx.draw_networkx(G,
                     pos=nx.spring_layout(G, seed=0),
                     with_labels=False,
                     node_size=150,
                     node_color=color,
                     width=0.8,
                     ax=ax[ix])

    classification_text = f"Predicted: {label_mapping[predicted_class]}, Actual: {label_mapping[actual_class]}"
    ax[ix].text(0.5, 0.9, classification_text, horizontalalignment='center', verticalalignment='center', transform=ax[ix].transAxes, color='black')

plt.show()


ModuleNotFoundError: No module named 'torch_geometric'